# Очистка и подготовка данных для обчения модели

In [31]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

import networkx as nx
import json
from sklearn.metrics.pairwise import cosine_similarity
import os
from collections import Counter, defaultdict

import seaborn as sns
from pyvis.network import Network
import re

import pickle

Соберем из json по папкам датасет с сылками на картинки, проставлеными таргетами, и другой полезной информацией для обучения модели.

In [32]:
start_dir = "data"

In [33]:
base_tags = [x[1] for x in os.walk(start_dir)][0] 

In [34]:
os.listdir(os.path.join(start_dir, base_tags[0]))

['101794343_571632827113069_4942026581500675390_n.jpg',
 '119805874_172751351093985_3145307797179512111_n.jpg',
 '119864372_195364978629260_4542001635666377025_n.jpg',
 '119889844_1336581619867268_6434906719318744210_n.jpg',
 '119891719_2761871797425200_2871559721718593590_n.jpg',
 '119940564_676867232923297_8440154950071313831_n.jpg',
 '119947020_434074184237289_126803323969749096_n.jpg',
 '119948616_608076269853785_3239600362009154538_n.jpg',
 '119971511_117251473298754_8027870046475495803_n.jpg',
 '119993916_178888233690747_4343367952865610120_n.jpg',
 '120042515_629730037744753_828134131595453103_n.jpg',
 '120043231_1707318749434044_7352190511072016974_n.jpg',
 '120127587_3109753125801484_8355312565610686899_n.jpg',
 '120657798_3297694073618130_6879107286593792447_n.jpg',
 '120734588_2421824494778313_539528843833307911_n.jpg',
 '120736331_3614794641875495_626853010276676780_n.jpg',
 '120740067_636261577279165_8735738333163088500_n.jpg',
 '120747100_2772780683048128_8096610674280200

In [35]:
def read_file(file_path):
    if file_path.endswith('.json'):
        with open(file_path, 'r', encoding='utf-8') as f:
            tmp = json.load(f)
        return tmp

In [36]:
data = list()
pattern = re.compile(r"(\d|_)*n.jpg")
for base_tag in tqdm(base_tags):
    PATH_TO_DATA = os.path.join(start_dir, base_tag) 
    list_files = os.listdir(PATH_TO_DATA)
    for json_file in os.listdir(PATH_TO_DATA):
            if not json_file.endswith('.json'):
                continue
            f = read_file(os.path.join(PATH_TO_DATA, json_file))

            # delete_bad_items(f['GraphImages'])
            for post_data in f['GraphImages']:
                if post_data['is_video']:
                    continue
                if len(post_data.get('tags', [])) == 0:
                    continue
                post_data = {key:post_data[key] for key in ['tags', 'urls', 'id', 'shortcode', 'edge_liked_by', 
                                                             'edge_media_to_comment', 'taken_at_timestamp']}    
#'comments_disabled',
                post_data['tags'] = [it.lower() for it in post_data['tags']]
                post_data['edge_liked_by'] = post_data['edge_liked_by']['count']
                post_data['edge_media_to_comment'] = post_data['edge_media_to_comment']['count']
            
                files = []
                line_data = post_data.copy()
                for i, url in enumerate(post_data['urls']):
                    m = pattern.search(url, 1)
                    if m is not None:
                        file = m.group(0)
                        if file in list_files:
                            line_data['base_tag'] = base_tag
                            line_data['sub_id'] = i
                            line_data['urls'] = file
                            data.append(line_data)
                        else:
                            pint(file)
                # post_data['urls'] = files
                # data.append(post_data)

len(data)

35316

In [37]:
df_data = pd.DataFrame(data)

In [38]:
df_data.head()

,tags,urls,id,shortcode,edge_liked_by,edge_media_to_comment,taken_at_timestamp,base_tag,sub_id
0,"[автомобили, ksipaoa123, photooftheday, home, ...",122743629_352858892655763_3862302602685435956_...,2430034908929605286,CG5N9_yjD6m,225,38,1603902756,авто,1
1,"[автомобили, ksipaoa123, photooftheday, home, ...",122743629_352858892655763_3862302602685435956_...,2430034908929605286,CG5N9_yjD6m,225,38,1603902756,авто,1
2,"[car, supercar, bikes, drivinghero, carinstagr...",122951186_986271131850482_7518795322651861682_...,2430094149860606462,CG5bcENMkH-,12921,39,1603909818,авто,0
3,"[car, supercar, bikes, drivinghero, carinstagr...",122961996_1509286229266836_4245020674540158164...,2429886929113785797,CG4sUm2MSXF,19543,52,1603885116,авто,2
4,"[car, supercar, bikes, drivinghero, carinstagr...",122961996_1509286229266836_4245020674540158164...,2429886929113785797,CG4sUm2MSXF,19543,52,1603885116,авто,2


In [39]:
arr = np.arange(df_data.shape[0])
idxs = np.random.permutation(arr)

In [40]:
df_data = df_data.iloc[idxs, :]

In [41]:
df_data.to_csv(os.path.join(start_dir, "data_set.csv"), index = False)

## Формируем словарик для категорий

In [24]:
category_id = {tag: i for i, tag  in enumerate(df_data.base_tag.unique())}

In [25]:
category_id

{'авто': 0,
 'девушка': 1,
 'дом': 2,
 'животные': 3,
 'здоровье': 4,
 'игрушка': 5,
 'искусство': 6,
 'косметика': 7,
 'красота': 8,
 'кухня': 9,
 'мебель': 10,
 'море': 11,
 'музыка': 12,
 'напитки': 13,
 'одежда': 14,
 'оружие': 15,
 'офис': 16,
 'праздник': 17,
 'растение': 18,
 'рыба': 19,
 'снаряжение': 20,
 'спорт': 21}

In [29]:
f = open(os.path.join(start_dir, "category_id.pkl"), 'wb')
pickle.dump(category_id, f)
f.close()